In [2]:
import torch
print(torch.__version__)
print(torch.version.cuda)         # None 表示 CPU-only
print(torch.cuda.is_available())  # False 表示没有可用 GPU

1.12.1+cpu
None
False


In [13]:
!python3 train.py --config configs/mnist_config.yaml --dataconfig configs/mnist_dataconfig.yaml

/home/dsand/miniconda3/envs/bm39/lib/python3.9/site-packages/monai/utils/module.py:367: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  pkg = __import__(module)  # top level module

*** Config file
configs/mnist_config.yaml

*** Dataconfig file
configs/mnist_dataconfig.yaml
/home/dsand/miniconda3/envs/bm39/lib/python3.9/site-packages/monai/utils/deprecate_utils.py:107: FutureWarning: <class 'monai.transforms.utility.array.AddChannel'>: Class `AddChannel` has been deprecated since version 0.8. please use MetaTensor data type and monai.transforms.EnsureChannelFirst instead.
  warn_deprecated(obj, msg, warning_category)
Running on cpu
100%|███████████████████████████████████████████| 10/10 [01:15<00:00,  7.52s/it]
train completed, best_metric: 0.8611 at epoch: 10
